# Sheet

In [1]:
#Importar librerias
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# VAMOS A CREAR UNA FUNCION QUE HAGA TODO EL PREPROCESADO TANTO PARA EL TRAIN COMO PARA EL TEST

In [2]:
def preprocess(train_df, test_df):

    import pandas as pd
    import numpy as np

    # ============================
    #   1. Trabajar con copias
    # ============================
    train = train_df.copy()
    test = test_df.copy()

    # Columnas que NO se deben eliminar para test antes de imputar
    drop_na_cols = [
        'E_PRGM_DEPARTAMENTO','INDICADOR_1', 'INDICADOR_2',
        'E_PRIVADO_LIBERTAD', 'RENDIMIENTO_GLOBAL', 'INDICADOR_3', 'INDICADOR_4'
    ]

    # Igualar columnas faltantes en test (por seguridad)
    for col in train.columns:
        if col not in test.columns:
            test[col] = np.nan

    # ============================
    #   2. Eliminar filas con NA en columnas clave (solo train)
    # ============================
    train = train.dropna(axis=0, subset=drop_na_cols)

    # ============================
    #   3. Definir funciones de imputación
    # ============================

    def imputar_valor_matricula(row):
        if pd.isna(row['E_VALORMATRICULAUNIVERSIDAD']):
            pago = row['E_PAGOMATRICULAPROPIO']
            estrato = row['F_ESTRATOVIVIENDA']

            if pago == 'No':
                return 'No pagó matrícula'
            if estrato == 'Estrato 1':
                return 'Menos de 500 mil'
            elif estrato == 'Estrato 2':
                return 'Entre 500 mil y menos de 1 millón'
            elif estrato == 'Estrato 3':
                return 'Entre 1 millón y menos de 2.5 millones'
            elif estrato in ['Estrato 4', 'Estrato 5']:
                return 'Entre 2.5 millones y menos de 4 millones'
            elif estrato == 'Estrato 6':
                return 'Más de 7 millones'
            else:
                return 'Entre 500 mil y menos de 1 millón'
        return row['E_VALORMATRICULAUNIVERSIDAD']

    def imputar_pago_por_estrato(row):
        if pd.isna(row['E_PAGOMATRICULAPROPIO']):
            estrato = row['F_ESTRATOVIVIENDA']
            if estrato in ['Estrato 1', 'Estrato 2', 'Estrato 3']:
                return 'No'
            elif estrato in ['Estrato 4', 'Estrato 5', 'Estrato 6']:
                return 'Si'
            else:
                return 'No'
        return row['E_PAGOMATRICULAPROPIO']

    def imputar_educacion(row, columna):
        valor = row[columna]
        estrato = row['F_ESTRATOVIVIENDA']
        otra_col = 'F_EDUCACIONMADRE' if columna == 'F_EDUCACIONPADRE' else 'F_EDUCACIONPADRE'

        if pd.notna(valor):
            return valor
        if pd.notna(row[otra_col]):
            return row[otra_col]

        if estrato in ['Estrato 1', 'Estrato 2']:
            return 'Primaria completa'
        elif estrato == 'Estrato 3':
            return 'Secundaria (Bachillerato) completa'
        elif estrato == 'Estrato 4':
            return 'Técnica o tecnológica completa'
        elif estrato in ['Estrato 5', 'Estrato 6']:
            return 'Educación profesional completa'
        return 'No sabe'

    def imputar_horas_trabaja(row):
        valor = row['E_HORASSEMANATRABAJA']
        if pd.notna(valor):
            return valor

        estrato = row['F_ESTRATOVIVIENDA']
        pago = row['E_PAGOMATRICULAPROPIO']
        edu_padre = row['F_EDUCACIONPADRE']
        edu_madre = row['F_EDUCACIONMADRE']

        if pago == 'Si' and estrato in ['Estrato 1', 'Estrato 2']:
            return 'Más de 30 horas'
        if pago == 'Si' and estrato == 'Estrato 3':
            return 'Entre 21 y 30 horas'
        if pago == 'Si' and estrato in ['Estrato 4', 'Estrato 5', 'Estrato 6']:
            return 'Menos de 10 horas'

        if pago == 'No' and (
            'profesional' in str(edu_padre).lower() or
            'postgrado' in str(edu_padre).lower() or
            'profesional' in str(edu_madre).lower() or
            'postgrado' in str(edu_madre).lower()
        ):
            return '0'

        if pago == 'No' and (
            'primaria' in str(edu_padre).lower() or
            'primaria' in str(edu_madre).lower() or
            'incompleta' in str(edu_padre).lower() or
            'incompleta' in str(edu_madre).lower()
        ):
            return 'Entre 11 y 20 horas'

        return '0'

    def imputar_internet_simple(row):
        valor = row['F_TIENEINTERNET']
        compu = row['F_TIENECOMPUTADOR']
        if pd.notna(valor):
            return valor
        if compu == 'Si':
            return 'Si'
        return 'No'

    def imputar_estrato(row):
        valor = row['F_ESTRATOVIVIENDA']
        edu_padre = str(row['F_EDUCACIONPADRE'])
        edu_madre = str(row['F_EDUCACIONMADRE'])
        if pd.notna(valor):
            return valor

        educaciones = (edu_padre + ' ' + edu_madre).lower()

        if any(n in educaciones for n in ['ninguno', 'primaria incompleta']):
            return 'Estrato 1'
        if any(n in educaciones for n in ['primaria completa', 'secundaria incompleta']):
            return 'Estrato 2'
        if any(n in educaciones for n in ['secundaria completa', 'técnica o tecnológica incompleta']):
            return 'Estrato 3'
        if any(n in educaciones for n in ['técnica o tecnológica completa', 'educación profesional incompleta']):
            return 'Estrato 4'
        if any(n in educaciones for n in ['educación profesional completa', 'postgrado']):
            return 'Estrato 5'
        return 'Estrato 2'

    def imputar_computador(row):
        valor = row['F_TIENECOMPUTADOR']
        estrato = row['F_ESTRATOVIVIENDA']
        edu_padre = str(row['F_EDUCACIONPADRE'])
        edu_madre = str(row['F_EDUCACIONMADRE'])
        internet = row['F_TIENEINTERNET']

        if pd.notna(valor):
            return valor
        if internet == 'Si':
            return 'Si'
        if estrato in ['Estrato 4', 'Estrato 5', 'Estrato 6']:
            return 'Si'

        educaciones = (edu_padre + ' ' + edu_madre).lower()
        if any(n in educaciones for n in ['técnica', 'tecnológica', 'profesional', 'universitaria', 'postgrado']):
            return 'Si'

        if estrato in ['Estrato 1', 'Estrato 2'] and any(
            n in educaciones for n in ['primaria', 'ninguno']
        ):
            return 'No'

        return 'No'

    # ==============================================
    #   4. Aplicar imputaciones a train y test
    # ==============================================
    for df in [train, test]:
        df['E_VALORMATRICULAUNIVERSIDAD'] = df.apply(imputar_valor_matricula, axis=1)
        df['E_PAGOMATRICULAPROPIO'] = df.apply(imputar_pago_por_estrato, axis=1)
        df['F_EDUCACIONPADRE'] = df.apply(lambda r: imputar_educacion(r, 'F_EDUCACIONPADRE'), axis=1)
        df['F_EDUCACIONMADRE'] = df.apply(lambda r: imputar_educacion(r, 'F_EDUCACIONMADRE'), axis=1)
        df['E_HORASSEMANATRABAJA'] = df.apply(imputar_horas_trabaja, axis=1)
        df['F_TIENEINTERNET'] = df.apply(imputar_internet_simple, axis=1)
        df['F_ESTRATOVIVIENDA'] = df.apply(imputar_estrato, axis=1)
        df['F_TIENECOMPUTADOR'] = df.apply(imputar_computador, axis=1)

    # ==============================
    #   5. Eliminar columnas
    # ==============================
    cols_drop = [
        'F_TIENEINTERNET.1', 'ID', 'PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO',
        'E_PRGM_DEPARTAMENTO', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL',
        'E_PRIVADO_LIBERTAD'
    ]

    train = train.drop(columns=[c for c in cols_drop if c in train.columns], errors='ignore')
    test = test.drop(columns=[c for c in cols_drop if c in test.columns], errors='ignore')

    # ==============================
    #   6. Codificación ordinal
    # ==============================
    orden_matricula = {
        'Menos de 500 mil': 1, 'Entre 500 mil y menos de 1 millón': 2,
        'Entre 1 millón y menos de 2.5 millones': 3, 'Entre 2.5 millones y menos de 4 millones': 4,
        'Entre 4 millones y menos de 5.5 millones': 5, 'Entre 5.5 millones y menos de 7 millones': 6,
        'Más de 7 millones': 7, 'No pagó matrícula': 0
    }

    orden_educacion = {
        'Ninguno': 0, 'Primaria incompleta': 1, 'Primaria completa': 2,
        'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
        'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
        'Educación profesional incompleta': 7, 'Educación profesional completa': 8,
        'Postgrado': 9, 'No sabe': -1, 'No Aplica': -1
    }

    orden_horas = {
        '0': 0, 'Menos de 10 horas': 1, 'Entre 11 y 20 horas': 2,
        'Entre 21 y 30 horas': 3, 'Más de 30 horas': 4
    }

    orden_estrato = {
        'Estrato 1': 1,'Estrato 2': 2,'Estrato 3': 3,
        'Estrato 4': 4,'Estrato 5': 5,'Estrato 6': 6,'Sin Estrato': 0
    }

    mapa_rendimiento = {'bajo': 1,'medio-bajo': 2,'medio-alto': 3,'alto': 4}

    # aplicar a ambos
    for df in [train, test]:
        df['E_VALORMATRICULAUNIVERSIDAD'] = df['E_VALORMATRICULAUNIVERSIDAD'].map(orden_matricula)
        df['F_EDUCACIONPADRE'] = df['F_EDUCACIONPADRE'].map(orden_educacion)
        df['F_EDUCACIONMADRE'] = df['F_EDUCACIONMADRE'].map(orden_educacion)
        df['E_HORASSEMANATRABAJA'] = df['E_HORASSEMANATRABAJA'].map(orden_horas)
        df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].map(orden_estrato)
        df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(mapa_rendimiento)

        df['F_TIENECOMPUTADOR'] = df['F_TIENECOMPUTADOR'].map({'No': 0, 'Si': 1})
        df['F_TIENEINTERNET'] = df['F_TIENEINTERNET'].map({'No': 0, 'Si': 1})
        df['E_PAGOMATRICULAPROPIO'] = df['E_PAGOMATRICULAPROPIO'].map({'No': 0, 'Si': 1})

    return train, test

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train, test = preprocess(train, test)

***Validemos que haya quedado bien!***

In [4]:
train.head(5)

,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,6,1,3,1,5,0,1,9,3,0.322,0.208,0.310,0.267
1,4,0,3,0,6,0,1,5,1,0.311,0.215,0.292,0.264
2,4,4,3,1,4,0,0,4,1,0.297,0.214,0.305,0.264
3,5,0,4,1,-1,0,1,4,4,0.485,0.172,0.252,0.190
4,4,3,3,1,2,0,1,2,2,0.316,0.232,0.285,0.294


In [5]:
test.head(5)

,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,RENDIMIENTO_GLOBAL
0,1,1,3,1,6,1,1,2,0.328,0.219,0.317,0.247,NaN
1,4,3,2,1,4,0,1,6,0.227,0.283,0.296,0.324,NaN
2,3,0,3,1,3,0,1,4,0.285,0.228,0.294,0.247,NaN
3,3,3,1,0,1,0,0,1,0.160,0.408,0.217,0.294,NaN
4,4,2,2,1,4,0,1,4,0.209,0.283,0.306,0.286,NaN


In [6]:
print(train.shape)
print(test.shape)

(692500, 13)
(296786, 13)


Aqui vemos que hay un problema, la funcion creo la columna RENDIMIENTO_GLOBAL en los datos del TEST, veamos que creo

In [7]:
test.columns

Index(['E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_EDUCACIONMADRE',
       'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4',
       'RENDIMIENTO_GLOBAL'],
      dtype='object')

Vemos que creo una columna llamada RENDIMIENTO_GLOBAL que es la variabla a predecir, esta columna no existia en el dataframe original asi deberia estar sin valores

In [8]:
test['RENDIMIENTO_GLOBAL']

0 NaN
1 NaN
2 NaN
3 NaN
4 NaN
 ..
296781 NaN
296782 NaN
296783 NaN
296784 NaN
296785 NaN
Name: RENDIMIENTO_GLOBAL, Length: 296786, dtype: float64

efectivamente no hay valores en esta por lo tanto no hay ***data lekage***, en ese orden de ideas eliminemos esa columna del test

In [9]:
test.drop('RENDIMIENTO_GLOBAL', axis=1, inplace=True)

In [10]:
#Ahora miremos el tamaño de nuestros datasets

print(train.shape)
print(test.shape)

(692500, 13)
(296786, 12)


In [11]:
print(train.columns)
print(test.columns)

Index(['E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_EDUCACIONMADRE',
       'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3',
       'INDICADOR_4'],
      dtype='object')
Index(['E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_EDUCACIONMADRE',
       'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4'],
      dtype='object')


In [12]:
#Separemos las variables predictotas de la variable objetivo

X = train.drop('RENDIMIENTO_GLOBAL', axis=1)
y = train['RENDIMIENTO_GLOBAL']

In [14]:
#Por Capacidad Computacional se hace una reduccion del datset de entrenamiento
from sklearn.model_selection import train_test_split

X_small, _, y_small, _ = train_test_split(
    X, y,
    train_size=0.1,       # 10% del dataset
    stratify=y,
    random_state=42
)

print(X_small.shape, y_small.shape)

(69250, 12) (69250,)


In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#Definir el modelo base
# Se inicializa el clasificador de bosque aleatorio
rf = RandomForestClassifier(random_state=42)

# 3. Definir la cuadrícula de hiperparámetros a buscar
# Define un diccionario con los parámetros y los valores se van probar.
param_grid = {
    'n_estimators': [50, 100],  # Número de árboles en el bosque
    'max_depth': [None, 10, 20], # Profundidad máxima de los árboles
    'min_samples_split': [2, 5], # Mínimo número de muestras requeridas para dividir un nodo
    'min_samples_leaf': [1, 2]   # Mínimo número de muestras requeridas en un nodo hoja
}

# 4. Configurar GridSearchCV
# cv=5 define la validación cruzada con 5 folds.
# n_jobs=-1 utiliza todos los núcleos de la CPU disponibles para paralelizar el entrenamiento
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=5, n_jobs=2, verbose=2, scoring='accuracy')

# 5. Ejecutar la búsqueda en la cuadrícula (esto tomará tiempo)
print("Iniciando Grid Search...")
grid_search.fit(X_small, y_small)
print("Grid Search completado.")

# 6. Mostrar los mejores parámetros y la mejor puntuación
print(f"Mejores parámetros encontrados: {grid_search.best_params_}")
print(f"Mejor puntuación (Accuracy CV): {grid_search.best_score_:.4f}")

# 7. Evaluar el mejor modelo en los datos de prueba
best_rf_model = grid_search.best_estimator_
y_pred = best_rf_model.predict(test)

Iniciando Grid Search...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Grid Search completado.
Mejores parámetros encontrados: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Mejor puntuación (Accuracy CV): 0.3854


In [17]:
y_pred

array([4, 3, 3, ..., 3, 4, 4])

In [18]:
d = pd.read_csv('test.csv')

In [20]:
id_column = d['ID']

df_kaggle = pd.DataFrame({
    'ID': id_column,
    'RENDIMIENTO_GLOBAL': y_pred
})

inverse_mapa_rendimiento = {
    1: 'bajo',
    2: 'medio-bajo',
    3: 'medio-alto',
    4: 'alto'
}

df_kaggle['RENDIMIENTO_GLOBAL'] = df_kaggle['RENDIMIENTO_GLOBAL'].map(inverse_mapa_rendimiento)

display(df_kaggle.head(5))

df_kaggle.to_csv('predicciones_rf_gs.csv', index=False)

,ID,RENDIMIENTO_GLOBAL
0,550236,alto
1,98545,medio-alto
2,499179,medio-alto
3,782980,bajo
4,785185,medio-bajo


# Sheet 2